In [3]:
from sklearn.neighbors import NearestNeighbors
import feedparser
import requests
from pymongo import MongoClient
import networkx as nx
from collections import defaultdict
import random
from sentence_transformers import SentenceTransformer, InputExample, losses
from torch.utils.data import DataLoader
from sentence_transformers.evaluation import InformationRetrievalEvaluator
import gensim

c:\Users\User\AppData\Local\Programs\Python\Python312\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [7]:
client = MongoClient("mongodb://localhost:27017/")
db = client["Arxiv"]
collection = db["Arxiv Papers"]

query = {"doi": { "$exists": True, "$ne": None }}
documents = collection.find(query, {"doi" : 1, "_id" : 0})
doi_list = [[doc["doi"]] for doc in documents]
#doi_list = doi_list[0:10] #slicing to reduce computational expensiveness

In [15]:
print(doi_list[0][0])

10.1175/WAF-D-22-0070.1


In [18]:
object = collection.find_one({"doi" : doi_list[0][0]})
update = {"$set" : {"citings" : []}}
print(object)

{'_id': ObjectId('679b07be98f57cd0508a3076'), 'doi': '10.1175/WAF-D-22-0070.1', 'title': 'a machine learning tutorial for operational meteorology, part i:\n  traditional machine learning', 'summary': "recently, the use of machine learning in meteorology has increased greatly. while many machine learning methods are not new, university classes on machine learning are largely unavailable to meteorology students and are not required to become a meteorologist. the lack of formal instruction has contributed to perception that machine learning methods are 'black boxes' and thus end-users are hesitant to apply the machine learning methods in their every day workflow. to reduce the opaqueness of machine learning methods and lower hesitancy towards machine learning in meteorology, this paper provides a survey of some of the most common machine learning methods. a familiar meteorological example is used to contextualize the machine learning methods while also discussing machine learning topics u

In [19]:
index = 0
for doi in doi_list:
    doi = doi[0]
    query = {"doi" : doi}
    print(index)
    url = f"https://opencitations.net/index/coci/api/v1/citations/{doi}"
    response = requests.get(url).json()
    arr = [citing["citing"] for citing in response]
    update = {"$set": {"citings" : arr}}
    collection.update_one(query, update)
    doi_list[index].append(arr)
    index += 1

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [ ]:
doi_list = doi_list[0:635]
doi_list = doi_list[0:len(doi_list) - 2]

Using mutual citations does not work since they are limited

In [53]:
# Extract all unique DOIs in the dataset
all_dois = {entry[0] for entry in doi_list}
citation_pairs = []
index = 0
for entry in doi_list:
    print(index)
    index += 1
    source_doi = entry[0]
    cited_dois = entry[1]
    
    for cited_doi in cited_dois:
        # Only include citations where the cited DOI exists in the dataset
        if cited_doi in all_dois:
            citation_pairs.append((source_doi, cited_doi))

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [54]:
mutual_citations = []

# Convert citation_pairs to a set for faster lookups
citation_set = set(citation_pairs)

for source, cited in citation_pairs:
    # Check if the reverse (cited, source) exists
    if (cited, source) in citation_set:
        mutual_citations.append((source, cited))

In [55]:
print(mutual_citations)

[('10.1002/cpe.7732', '10.1002/cpe.7732'), ('10.1145/3365365.3383468', '10.1145/3365365.3383468'), ('10.1063/5.0065093', '10.1063/5.0065093'), ('10.1080/15376494.2022.2102274', '10.1080/15376494.2022.2102274')]


Relationships based on direction citations, co-citations or shared references may be better

In [73]:
print(doi_list[0])

['10.1175/WAF-D-22-0070.1', ['10.3390/environsciproc2023026103', '10.1007/978-3-031-35894-4_42', '10.5194/gmd-17-911-2024', '10.1002/minf.202300210', '10.1007/s42380-023-00191-9', '10.3390/meteorology3010003', '10.1007/s13351-023-2146-2', '10.3389/feart.2022.1089112', '10.1007/978-3-031-43085-5_23', '10.1038/s43017-023-00468-z', '10.1039/d3se00457k'], ['10.3390/environsciproc2023026103', '10.1007/978-3-031-35894-4_42', '10.5194/gmd-17-911-2024', '10.1002/minf.202300210', '10.1007/s42380-023-00191-9', '10.3390/meteorology3010003', '10.1007/s13351-023-2146-2', '10.3389/feart.2022.1089112', '10.1007/978-3-031-43085-5_23', '10.1038/s43017-023-00468-z', '10.1039/d3se00457k']]


In [74]:
def remove_duplicate_citations(doi_list):
    for item in doi_list:
        doi = item[0]  # The main DOI
        citation_lists = item[1:]  # All citation lists

        # Flatten the citation lists into a single list
        all_citations = [citation for sublist in citation_lists for citation in sublist]

        # Remove duplicates using a set
        unique_citations = list(set(all_citations))

        # Reconstruct the item with unique citations
        item[1:] = [unique_citations]  # Replace all citation lists with a single list of unique citations

    return doi_list

In [75]:
cleaned_doi = remove_duplicate_citations(doi_list)

In [77]:
print(cleaned_doi[0])

['10.1175/WAF-D-22-0070.1', ['10.1038/s43017-023-00468-z', '10.3390/meteorology3010003', '10.3390/environsciproc2023026103', '10.1039/d3se00457k', '10.5194/gmd-17-911-2024', '10.3389/feart.2022.1089112', '10.1007/978-3-031-43085-5_23', '10.1007/s13351-023-2146-2', '10.1007/978-3-031-35894-4_42', '10.1007/s42380-023-00191-9', '10.1002/minf.202300210']]


In [78]:
positive_pairs = []

for source, cited_dois in cleaned_doi:
    for cited in cited_dois:
        positive_pairs.append((source, cited))

print(positive_pairs)

[('10.1175/WAF-D-22-0070.1', '10.1038/s43017-023-00468-z'), ('10.1175/WAF-D-22-0070.1', '10.3390/meteorology3010003'), ('10.1175/WAF-D-22-0070.1', '10.3390/environsciproc2023026103'), ('10.1175/WAF-D-22-0070.1', '10.1039/d3se00457k'), ('10.1175/WAF-D-22-0070.1', '10.5194/gmd-17-911-2024'), ('10.1175/WAF-D-22-0070.1', '10.3389/feart.2022.1089112'), ('10.1175/WAF-D-22-0070.1', '10.1007/978-3-031-43085-5_23'), ('10.1175/WAF-D-22-0070.1', '10.1007/s13351-023-2146-2'), ('10.1175/WAF-D-22-0070.1', '10.1007/978-3-031-35894-4_42'), ('10.1175/WAF-D-22-0070.1', '10.1007/s42380-023-00191-9'), ('10.1175/WAF-D-22-0070.1', '10.1002/minf.202300210'), ('10.1007/978-1-4615-5529-2', '10.3390/app13010626'), ('10.1007/978-1-4615-5529-2', '10.1007/978-3-030-64096-5_3'), ('10.1007/978-1-4615-5529-2', '10.1002/icd.544'), ('10.1007/978-1-4615-5529-2', '10.1142/s0219843608001558'), ('10.1007/978-1-4615-5529-2', '10.1007/s10115-013-0665-3'), ('10.1007/978-1-4615-5529-2', '10.1007/978-981-19-7742-8_5'), ('10.100

Train model based on these positive pairs

First we generate negative pairs

In [79]:
all_citations = []
for source, citations in cleaned_doi:
        for i in range(len(citations)):
                all_citations.append(citations[i])


In [80]:
print(all_citations)

['10.1038/s43017-023-00468-z', '10.3390/meteorology3010003', '10.3390/environsciproc2023026103', '10.1039/d3se00457k', '10.5194/gmd-17-911-2024', '10.3389/feart.2022.1089112', '10.1007/978-3-031-43085-5_23', '10.1007/s13351-023-2146-2', '10.1007/978-3-031-35894-4_42', '10.1007/s42380-023-00191-9', '10.1002/minf.202300210', '10.3390/app13010626', '10.1007/978-3-030-64096-5_3', '10.1002/icd.544', '10.1142/s0219843608001558', '10.1007/s10115-013-0665-3', '10.1007/978-981-19-7742-8_5', '10.1109/access.2017.2696365', '10.1109/cic.2003.1291159', '10.1145/3609483', '10.1075/ijlcr.18012.bal', '10.1109/icassp40776.2020.9054694', '10.1109/tmm.2020.3001510', '10.1007/3-540-44668-0_13', '10.1002/ange.202101986', '10.1007/s10994-008-5050-1', '10.1145/1014052.1014067', '10.1017/9781108755610.014', '10.34133/2021/9807452', '10.1049/ipr2.12889', '10.1109/icmla.2011.16', '10.1007/978-3-540-39907-0_35', '10.1109/wacv45572.2020.9093536', '10.1109/imcsit.2009.5352802', '10.1541/ieejeiss.123.1004', '10.110

In [81]:
negative_examples = []

for source, citation in cleaned_doi:
    negative_examples.append([source])
    choices = []
    for i in range(0, 10):
        choice = random.choice(all_citations)
        while choice in citation:
            choice = random.choice(all_citations)
        choices.append(choice)
    negative_examples[-1].append(choices)


In [82]:
negative_pairs = []
for base, negative in negative_examples:
    for negatives in negative:
        negative_pairs.append((base, negatives))


In [83]:
print(positive_pairs[0])

('10.1175/WAF-D-22-0070.1', '10.1038/s43017-023-00468-z')


In [84]:
training_data = []
for positive in positive_pairs:
    training_data.append(InputExample(texts=[positive[0],positive[1]], label= 1))
for negative in negative_pairs:
    training_data.append(InputExample(texts=[negative[0],negative[1]], label= 0))

In [85]:
model = SentenceTransformer('all-MiniLM-L6-v2')
model = model.to('cuda')

c:\Users\User\AppData\Local\Programs\Python\Python312\Lib\site-packages\huggingface_hub\file_download.py:795: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [86]:
train_dataloader = DataLoader(training_data, shuffle= True, batch_size= 32)
train_loss = losses.CosineSimilarityLoss(model)

Now we fine-tune the model accordingly

In [87]:
model.fit(train_objectives=[(train_dataloader, train_loss)], epochs= 5, warmup_steps=100, optimizer_params= {"lr": 2e-5}, output_path="./fine_tuned", show_progress_bar= True)

  0%|          | 0/6160 [00:00<?, ?it/s]

{'loss': 0.1294, 'grad_norm': 0.4609180688858032, 'learning_rate': 1.8678777869529318e-05, 'epoch': 0.41}
{'loss': 0.1155, 'grad_norm': 0.5118800401687622, 'learning_rate': 1.702725020644096e-05, 'epoch': 0.81}
{'loss': 0.1094, 'grad_norm': 0.6851925253868103, 'learning_rate': 1.5375722543352604e-05, 'epoch': 1.22}
{'loss': 0.0987, 'grad_norm': 0.6106853485107422, 'learning_rate': 1.3724194880264245e-05, 'epoch': 1.62}
{'loss': 0.0984, 'grad_norm': 1.1688690185546875, 'learning_rate': 1.2072667217175889e-05, 'epoch': 2.03}
{'loss': 0.0928, 'grad_norm': 0.8557384610176086, 'learning_rate': 1.0421139554087532e-05, 'epoch': 2.44}
{'loss': 0.0907, 'grad_norm': 0.9869582056999207, 'learning_rate': 8.769611890999175e-06, 'epoch': 2.84}
{'loss': 0.0883, 'grad_norm': 0.8429928421974182, 'learning_rate': 7.118084227910817e-06, 'epoch': 3.25}
{'loss': 0.0842, 'grad_norm': 1.0029890537261963, 'learning_rate': 5.466556564822461e-06, 'epoch': 3.65}
{'loss': 0.0843, 'grad_norm': 1.2839471101760864, 

Computing widget examples:   0%|          | 0/1 [00:00<?, ?example/s]

We can vindicate performance using InformationRetrievalEvaluator, after this we use the model to change the embeddings of each arxiv article

After changing the embeddings we push this into the recommendation pipeline (start with something simple like KNN)

In [88]:
model = SentenceTransformer("./fine_tuned")